In [4]:
import torch
from torch.utils.data import SubsetRandomSampler, DataLoader, RandomSampler
import torchvision
from torchvision import datasets, transforms
import os
from inclearn.lib import data, factory, losses, network, utils
import clip
import numpy as np

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:0 device


# This is base code

In [6]:
model, preprocess = clip.load('ViT-B/32', device=device)
out_dim = model.text_projection.shape[1]

print("Features dimension is {}.".format(out_dim))

model.to(device)
model.eval()
torch.cuda.empty_cache()

Features dimension is 512.


In [7]:
class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]


cifar100_train = torchvision.datasets.CIFAR100(root="./data", train=True, transform=preprocess)
cifar100_test = torchvision.datasets.CIFAR100(root="./data", train=False, transform=preprocess)

train_list = {}
test_list = {}
for i in range(100):
    train_list[i] = []
    test_list[i] = []
for idx, (X,y) in enumerate(cifar100_train):
    train_list[y].append(idx)    
for idx, (X,y) in enumerate(cifar100_test):
    test_list[y].append(idx)

In [10]:
torch.save(train_list,'./exps/zero_train/train_list.pth')

In [11]:
torch.save(test_list,'./exps/zero_train/test_list.pth')

In [ ]:
feature_means = np.zeros((100,512))
for class_idx in class_order:
    class_loader = DataLoader(cifar100_train, shuffle=False, batch_size=500, 
                            sampler=SubsetRandomSampler(train_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.to(device)

            features = model.encode_image(X).cpu().numpy()
            feature_means[class_idx] = features.mean(axis=0)

        torch.cuda.empty_cache()

In [ ]:
class_acc = []

for class_idx in class_order:
    class_loader = DataLoader(cifar100_test, shuffle=False, batch_size=100, 
                            sampler=SubsetRandomSampler(test_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.numpy()

            features = model.encode_image(X).cpu().numpy()
            
            dists = -2 * (features@feature_means.T) + np.power(features, 2).sum(axis=1, keepdims=True) + np.power(feature_means, 2).sum(axis=1, keepdims=True).T
            preds = dists.argsort()[:,0]
            class_acc.append(np.sum(preds == y) / 100)

print(class_acc)
print(np.mean(class_acc))

# Zero trained CLIP model accuracy function

In [5]:
def get_zero_acc(model_name):
    model, preprocess = clip.load(model_name, device=device)
    out_dim = model.text_projection.shape[1]

    print("Features dimension is {}.".format(out_dim))

    model.to(device)
    model.eval()
    torch.cuda.empty_cache()

    class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]

    cifar100_train = torchvision.datasets.CIFAR100(root="./data", train=True, transform=preprocess)
    cifar100_test = torchvision.datasets.CIFAR100(root="./data", train=False, transform=preprocess)

    train_list = {}
    test_list = {}
    for i in range(100):
        train_list[i] = []
        test_list[i] = []
    for idx, (X,y) in enumerate(cifar100_train):
        train_list[y].append(idx)    
    for idx, (X,y) in enumerate(cifar100_test):
        test_list[y].append(idx)
    
    feature_means = np.zeros((100,out_dim))
    for class_idx in class_order:
        class_loader = DataLoader(cifar100_train, shuffle=False, batch_size=500, 
                                sampler=SubsetRandomSampler(train_list[class_idx]))
        with torch.no_grad():
            for X, y in class_loader:
                X, y = X.to(device), y.to(device)

                features = model.encode_image(X).cpu().numpy()
                feature_means[class_idx] = features.mean(axis=0)

            torch.cuda.empty_cache()
    
    class_acc = []

    for class_idx in class_order:
        class_loader = DataLoader(cifar100_test, shuffle=False, batch_size=100, 
                                sampler=SubsetRandomSampler(test_list[class_idx]))
        with torch.no_grad():
            for X, y in class_loader:
                X, y = X.to(device), y.numpy()

                features = model.encode_image(X).cpu().numpy()
                
                dists = -2 * (features@feature_means.T) + np.power(features, 2).sum(axis=1, keepdims=True) + np.power(feature_means, 2).sum(axis=1, keepdims=True).T
                preds = dists.argsort()[:,0]
                class_acc.append(np.sum(preds == y) / 100)

    print(class_acc)
    print(np.mean(class_acc))

In [3]:
for model_name in clip.available_models():
    print(model_name)
    # get_zero_acc(model_name)
    print()

RN50

RN101

RN50x4

RN50x16

RN50x64

ViT-B/32

ViT-B/16

ViT-L/14

ViT-L/14@336px



# Accuracy to Average Accuracy

CIL에서 주로 가져오는 metric이 average accuracy인데, 지금 뽑아놓은 숫자가 그냥 accuracy에 해당하기 때문에 average accuracy화 해줘야 한다.

In [10]:
import numpy as np

def get_average_accuracy(acc,inc):
    accs = []
    for i in range(50//inc+1):
        inc_acc = np.mean(acc[:50+i*inc])
        accs.append(inc_acc)

    return np.mean(accs), accs[-1]

In [16]:
acc = [0.64, 0.76, 0.69, 0.76, 0.15, 0.37, 0.7, 0.3, 0.38, 0.31, 0.65, 0.32, 0.41, 0.16, 0.77, 0.42, 0.53, 0.39, 0.56, 0.7, 0.37, 0.3, 0.46, 0.63, 0.21, 0.33, 0.23, 0.45, 0.12, 0.31, 0.53, 0.13, 0.35, 0.37, 0.39, 0.12, 0.46, 0.18, 0.6, 0.46, 0.71, 0.58, 0.55, 0.58, 0.67, 0.32, 0.15, 0.75, 0.15, 0.42, 0.55, 0.62, 0.37, 0.68, 0.47, 0.74, 0.76, 0.72, 0.39, 0.38, 0.32, 0.3, 0.55, 0.08, 0.42, 0.54, 0.69, 0.25, 0.65, 0.57, 0.76, 0.49, 0.61, 0.55, 0.62, 0.62, 0.4, 0.33, 0.18, 0.65, 0.39, 0.42, 0.69, 0.59, 0.56, 0.41, 0.55, 0.76, 0.78, 0.24, 0.63, 0.5, 0.71, 0.19, 0.31, 0.17, 0.7, 0.27, 0.47, 0.76]

for inc in [10,5,2,1]:
    print("Incremental Class Number :", inc)
    avg_acc, last_acc = get_average_accuracy(acc,inc)
    print("    Average Accuracy :", avg_acc)
    print("    Last Accuracy    :", last_acc)

Incremental Class Number : 10
    Average Accuracy : 0.4599574735449736
    Last Accuracy    : 0.47209999999999996
Incremental Class Number : 5
    Average Accuracy : 0.46005958799073776
    Last Accuracy    : 0.47209999999999996
Incremental Class Number : 2
    Average Accuracy : 0.46065413325291826
    Last Accuracy    : 0.47209999999999996
Incremental Class Number : 1
    Average Accuracy : 0.46095812526214125
    Last Accuracy    : 0.47209999999999996
